In [1]:
import os
import numpy as np
import pickle
import pandas as pd 

from tqdm import tqdm

In [2]:
dir = [d for d in os.listdir('../experiments') if d!='logs']

In [3]:
methods = ['Upsampling', 'SMOTE', 'ADASYN', 'BORDELINE']

In [4]:
def generate_test(sample, thr_bas_default, thr_aug_default):
    results = []
    for d in dir:
        path_main = '../experiments/' + d
        for method in methods:
            pct_gain_list = []
            for idx in tqdm(range(50)):
                base_metrics = pickle.load(open(path_main + f'/metrics_base_{sample}_{idx}.pkl', 'rb'))
                method_metrics = pickle.load(open(path_main + f'/{method}/target_0.5_{sample}_{idx}.pkl', 'rb'))
                ba_list = []
                for k in range(len(base_metrics)):
                    if thr_bas_default:
                        tn, fp, fn, tp = base_metrics[k]['matrix'][.5].ravel()
                    else:
                        thr = base_metrics[k]['thr']
                        tn, fp, fn, tp = base_metrics[k]['matrix'][thr].ravel()
                    ba = (tp/(tp+fn) + tn/(tn+fp))/2
                    ba_list.append(ba)
                ba_model = []
                for k in range(len(method_metrics)):
                    if thr_aug_default:
                        tn, fp, fn, tp = method_metrics[k]['matrix'][.5].ravel()
                    else:
                        thr = method_metrics[k]['thr']
                        tn, fp, fn, tp = method_metrics[k]['matrix'][thr].ravel()
                    ba = (tp/(tp+fn) + tn/(tn+fp))/2
                    ba_model.append(ba)
                pct_gain = (np.array(ba_model) - np.mean(ba_list))/(np.mean(ba_list)) * 100
                pct_gain_list.append(pct_gain)
            pct_gain_list = np.array(pct_gain_list)
            mu_j = pct_gain_list.mean(axis=1)
            sigma_j = pct_gain_list.std(axis=1)
            sigma = pct_gain_list.mean(axis=1).std()

            sample_pct_mean = []
            print('Começando bootstrap')
            for B in range(1000):
                sample_pct = []
                sample_mu = np.random.normal(0, sigma, 50)
                for j in range(50):
                    sample_mu_j = sample_mu[j]
                    sample_pct_j = np.random.normal(sample_mu_j, sigma_j[j], 40)
                    sample_pct.extend(sample_pct_j.tolist())
                sample_pct_mean.append(np.mean(sample_pct))
            p_value = np.mean(np.abs(sample_pct_mean) > np.abs(pct_gain_list.mean()))
            results.append([d, method, pct_gain_list.mean(), np.median(pct_gain_list), p_value])
    return results

In [5]:
list_df = []
sample = 500
results = generate_test(sample, True, True)
df = pd.DataFrame(results, columns=['dataset', 'method', 'mean', 'median', 'p_value'])

# Cria uma nova coluna 'mean_p' que é igual à coluna 'mean' onde 'p_value' < 0.01, e NaN caso contrário.
df['mean'] = df['mean'].round()
df.loc[df['mean']==0, 'mean'] = 0
df['mean_p'] = np.where(df['p_value'] < 0.01, df['mean'], np.nan)
df['mean_text'] = np.where(df['p_value'] < 0.01, df['mean'].astype(str), df['mean'].apply(lambda x:f'{x}*'))
df['facet_a'] = 'c=0.5 for both model'
df['facet_b'] = f'n={sample}'
list_df.append(df)

results = generate_test(sample, False, True)
df = pd.DataFrame(results, columns=['dataset', 'method', 'mean', 'median', 'p_value'])
df['mean'] = df['mean'].round()
df.loc[df['mean']==0, 'mean'] = 0
df['mean_p'] = np.where(df['p_value'] < 0.01, df['mean'], np.nan)
df['mean_text'] = np.where(df['p_value'] < 0.01, df['mean'].astype(str), df['mean'].apply(lambda x:f'{x}*'))
df['facet_a'] = 'c=0.5 for augmented and \n optimized for base model'
df['facet_b'] = f'n={sample}'
list_df.append(df)


results = generate_test(sample, False, False)
df = pd.DataFrame(results, columns=['dataset', 'method', 'mean', 'median', 'p_value'])

df['mean'] = df['mean'].round()
df.loc[df['mean']==0, 'mean'] = 0
df['mean_p'] = np.where(df['p_value'] < 0.01, df['mean'], np.nan)
df['mean_text'] = np.where(df['p_value'] < 0.01, df['mean'].astype(str), df['mean'].apply(lambda x:f'{x}*'))
df['facet_a'] = 'Optimized c for both model'
df['facet_b'] = f'n={sample}'
list_df.append(df)


sample = 2000
results = generate_test(sample, True, True)
df = pd.DataFrame(results, columns=['dataset', 'method', 'mean', 'median', 'p_value'])

df['mean'] = df['mean'].round()
df.loc[df['mean']==0, 'mean'] = 0
df['mean_p'] = np.where(df['p_value'] < 0.01, df['mean'], np.nan)
df['mean_text'] = np.where(df['p_value'] < 0.01, df['mean'].astype(str), df['mean'].apply(lambda x:f'{x}*'))
df['facet_a'] = 'c=0.5 for both model'
df['facet_b'] = f'n={sample}'
list_df.append(df)

results = generate_test(sample, False, True)
df = pd.DataFrame(results, columns=['dataset', 'method', 'mean', 'median', 'p_value'])
df['mean'] = df['mean'].round()
df.loc[df['mean']==0, 'mean'] = 0
df['mean_p'] = np.where(df['p_value'] < 0.01, df['mean'], np.nan)
df['mean_text'] = np.where(df['p_value'] < 0.01, df['mean'].astype(str), df['mean'].apply(lambda x:f'{x}*'))
df['facet_a'] = 'c=0.5 for augmented and \n optimized for base model'
df['facet_b'] = f'n={sample}'
list_df.append(df)


results = generate_test(sample, False, False)
df = pd.DataFrame(results, columns=['dataset', 'method', 'mean', 'median', 'p_value'])

df['mean'] = df['mean'].round()
df.loc[df['mean']==0, 'mean'] = 0
df['mean_p'] = np.where(df['p_value'] < 0.01, df['mean'], np.nan)
df['mean_text'] = np.where(df['p_value'] < 0.01, df['mean'].astype(str), df['mean'].apply(lambda x:f'{x}*'))
df['facet_a'] = 'Optimized c for both model'
df['facet_b'] = f'n={sample}'
list_df.append(df)



100%|██████████| 50/50 [00:01<00:00, 39.58it/s]


Começando bootstrap


100%|██████████| 50/50 [00:01<00:00, 43.83it/s]


Começando bootstrap


100%|██████████| 50/50 [00:01<00:00, 44.72it/s]


...

Começando bootstrap


100%|██████████| 50/50 [00:01<00:00, 30.98it/s]


Começando bootstrap


100%|██████████| 50/50 [00:01<00:00, 43.42it/s]


Começando bootstrap


In [8]:
path_main = '../experiments/churn'
sample = '500'
method = 'Upsampling'
idx = 0

base_metrics = pickle.load(open(path_main + f'/metrics_base_{sample}_{idx}.pkl', 'rb'))
method_metrics = pickle.load(open(path_main + f'/{method}/target_0.5_{sample}_{idx}.pkl', 'rb'))

In [11]:
base_metrics[0]['thr']

0.78

In [9]:
method_metrics[0]['thr']

0.66

In [37]:
df_heatmap = pd.concat(list_df)

In [38]:
df_heatmap.to_csv('heatmap_ba.csv')